In [1]:
# import everything you need
from nipype import Node, Workflow
import nipype.interfaces.spm as spm
from nipype.interfaces.base import CommandLine
from nilearn import image as nlimg
from nilearn import masking as nlmask
from nilearn.plotting import plot_anat, plot_epi, plot_roi
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nibabel as nb
import os, re
from itertools import zip_longest

200220-14:48:56,84 nipype.utils INFO:
	 Running nipype version 1.5.0-dev (latest: 1.4.2)


Gather necessary pre-requisites:

In [2]:
# define useful functions
# define show-slices function
def show_slices(slices):
    """ Function to display row of image slices """
    fig, axes = plt.subplots(1, len(slices))
    for i, slice in enumerate(slices):
        axes[i].imshow(slice.T, cmap="gray", origin="lower")
# add suptitle via plt.suptitle
        
# check size of items and delete them when above a certain threshold
from sys import getsizeof # function is not fully indicative -> look for better option
def check_itemsize(items):
    for i in items:
        # return size of object in bytes
        print("size of item", i, "is:", getsizeof(i))
        if getsizeof(i) > 10000: # think about dynamic coding for this
            del i
            print("deleted, object exceeded set threshold.")
        else:
            print("retained, object is sufficiently small.")

In [3]:
# calculate "global values" via nibabel objects: sum up the values and multiply by volume of each voxel
# from: total volume GM, WM and CSF

# find all the files starting with "c1,c2,c3" in all subdirs of the VBM directory via CommandLine interface from nipype
c1_files=CommandLine('find', args='/VBM/FED* -maxdepth 2 -type f -name c1*.nii', terminal_output='allatonce')
c1=c1_files.run()
c2_files=CommandLine('find', args='/VBM/FED* -maxdepth 2 -type f -name c2*.nii', terminal_output='allatonce')
c2=c2_files.run()
c3_files=CommandLine('find', args='/VBM/FED* -maxdepth 2 -type f -name c3*.nii', terminal_output='allatonce')
c3=c3_files.run()

# read the output (find stdout) line by line to list object
globals_GM=c1.runtime.stdout.splitlines()
globals_WM=c2.runtime.stdout.splitlines()
globals_CSF=c3.runtime.stdout.splitlines()

# remove subject 56 from lists(-> was not normalised or in template)
globals_GM=[i for i in globals_GM if i.rsplit('/',3)[1] != "FED056"]
globals_WM=[i for i in globals_WM if i.rsplit('/',3)[1] != "FED056"]
globals_CSF=[i for i in globals_CSF if i.rsplit('/',3)[1] != "FED056"]

# create nibabel objects from all files
globals_GM_nb=[nb.load(i) for i in globals_GM]
globals_WM_nb=[nb.load(i) for i in globals_WM]
globals_CSF_nb=[nb.load(i) for i in globals_CSF]

# create list with the func data via dataobj property (avoids caching entire object (.fdata()) but gets func data as well)
globals_GM_fvals=[i.dataobj for i in globals_GM_nb]
globals_WM_fvals=[i.dataobj for i in globals_WM_nb]
globals_CSF_fvals=[i.dataobj for i in globals_CSF_nb]

In [4]:
print(globals_GM)

['/VBM/FED006/DARTEL_newsegment/c1CON_T1_MPRAGE_SAG_ISO_0_9_0005_20141106153429.nii', '/VBM/FED007/DARTEL_newsegment/c1CON_T1_MPRAGE_SAG_ISO_0_9_0005_20141107104226.nii', '/VBM/FED008/DARTEL_newsegment/c1CON_T1_MPRAGE_SAG_ISO_0_9_0005_20141112155009.nii', '/VBM/FED009/DARTEL_newsegment/c1CON_T1_MPRAGE_SAG_ISO_0_9_0005_20141117154954.nii', '/VBM/FED010/DARTEL_newsegment/c1CON_T1_MPRAGE_SAG_ISO_0_9_0005_20141119091904.nii', '/VBM/FED011/DARTEL_newsegment/c1DEP_T1_MPRAGE_SAG_ISO_0_9_0005_20141119161708.nii', '/VBM/FED012/DARTEL_newsegment/c1CON_T1_MPRAGE_SAG_ISO_0_9_0005_20141124150148.nii', '/VBM/FED013/DARTEL_newsegment/c1CON_T1_MPRAGE_SAG_ISO_0_9_0005_20141125093955.nii', '/VBM/FED014/DARTEL_newsegment/c1DEP_T1_MPRAGE_SAG_ISO_0_9_0005_20141125153216.nii', '/VBM/FED015/DARTEL_newsegment/c1CON_T1_MPRAGE_SAG_ISO_0_9_0005_20141202145526.nii', '/VBM/FED016/DARTEL_newsegment/c1CON_T1_MPRAGE_SAG_ISO_0_9_0005_20141202164821.nii', '/VBM/FED017/DARTEL_newsegment/c1CON_T1_MPRAGE_SAG_ISO_0_9_0005_

In [4]:
# 0 values outside the brain in each volume -> 0 vals overbias and make globals smaller
# c1,2,3 also are not normalised -> different positions for different subjects
# -> take all c images and calculate below values for each matter ;)

# reasign lists
globals_GM=[]
globals_WM=[]
globals_CSF=[]

# loop over GM WM CSF lists and calculate globals
for a,b in zip_longest(globals_GM_fvals,globals_GM_nb):
    globals_GM.append((np.sum(a[:,:,:][a[:,:,:] != 0])
                       * np.product(b.header['pixdim'][1:4]))
                       / np.size(a[:,:,:][a[:,:,:] != 0]))

for a,b in zip_longest(globals_WM_fvals,globals_WM_nb):
    globals_WM.append((np.sum(a[:,:,:][a[:,:,:] != 0])
                       * np.product(b.header['pixdim'][1:4]))
                       / np.size(a[:,:,:][a[:,:,:] != 0]))

for a,b in zip_longest(globals_CSF_fvals,globals_CSF_nb):
    globals_CSF.append((np.sum(a[:,:,:][a[:,:,:] != 0])
                        * np.product(b.header['pixdim'][1:4]))
                        / np.size(a[:,:,:][a[:,:,:] != 0]))

# combine globals to get total intracranial volume (TiCV)
globals_TiCV=[]

for a,b,c in zip_longest(globals_GM,globals_WM,globals_CSF):
    globals_TiCV.append(np.sum([a,b,c]))
    
# establish range of values
#for a,b,c in zip_longest(globals_GM_fvals, globals_WM_fvals, globals_CSF_fvals):
#    print(np.amin(a), np.amax(a),np.amin(b), np.amax(b),np.amin(c), np.amax(c))
# always the same: 0.0 - 1.0000000591389835

In [5]:
# get normalised GM/WM images to analyse in MultipleRegression model
# change norm_dir to vary between GM and WM analysis
norm_dir="/VBM/DARTEL_norm2mni_c1/"
images_mreg=[]

for i in os.listdir(norm_dir):
    match=re.match(r'(smwc.*\d{14,}.*.nii)', i)
    if match:
        images_mreg.append(os.path.join(norm_dir, match.group(0)))

#print("Images where normalised only if included in study specific template!")
#print(images_mreg, len(images_mreg))

# group them correctly
dep_norm=[]
con_norm=[]

for i in images_mreg:
    match_dep=re.match(r'.*DEP_.*.nii', i)
    match_con=re.match(r'.*CON_.*.nii', i)
    if match_dep:
        dep_norm.append(match_dep.group(0))
    elif match_con:
        con_norm.append(match_con.group(0))
    else:
        print("item in images_mreg does not match regex pattern! Investigate ... ")
    
print(dep_norm, len(dep_norm))
print(con_norm, len(con_norm))

['/VBM/DARTEL_norm2mni_c1/smwc1DEP_T1_MPRAGE_SAG_ISO_0_9_0005_20151216154455.nii', '/VBM/DARTEL_norm2mni_c1/smwc1DEP_T1_MPRAGE_SAG_ISO_0_9_0005_20151001173950.nii', '/VBM/DARTEL_norm2mni_c1/smwc1DEP_T1_MPRAGE_SAG_ISO_0_9_0005_20150929162030.nii', '/VBM/DARTEL_norm2mni_c1/smwc1DEP_T1_MPRAGE_SAG_ISO_0_9_0005_20151009111760.nii', '/VBM/DARTEL_norm2mni_c1/smwc1DEP_T1_MPRAGE_SAG_ISO_0_9_0005_20150219142148.nii', '/VBM/DARTEL_norm2mni_c1/smwc1DEP_T1_MPRAGE_SAG_ISO_0_9_0005_20141208163153.nii', '/VBM/DARTEL_norm2mni_c1/smwc1DEP_T1_MPRAGE_SAG_ISO_0_9_0005_20151001125857.nii', '/VBM/DARTEL_norm2mni_c1/smwc1DEP_T1_MPRAGE_SAG_ISO_0_9_0005_20150313125246.nii', '/VBM/DARTEL_norm2mni_c1/smwc1DEP_T1_MPRAGE_SAG_ISO_0_9_0005_20151015122011.nii', '/VBM/DARTEL_norm2mni_c1/smwc1DEP_T1_MPRAGE_SAG_ISO_0_9_0005_20150917120306.nii', '/VBM/DARTEL_norm2mni_c1/smwc1DEP_T1_MPRAGE_SAG_ISO_0_9_0005_20150310152650.nii', '/VBM/DARTEL_norm2mni_c1/smwc1DEP_T1_MPRAGE_SAG_ISO_0_9_0005_20160310171152.nii', '/VBM/DARTEL_no

In [6]:
# create covariates (SPM makes NO DIFFERENCE btw covariates and conditions)
# get depression, sex and age - data from list in .xlsx file 
cov_file_ID="FED_Subject_Assignments.xlsx"
content=pd.read_excel(cov_file_ID, sheet_name="Sheet2")

# remove subject 56 from lists(-> was not normalised or in template)
content=content[content["Sub Num FED_XXX"] != 56]
depression=content['BDI 22 Score'].tolist()
# reverse-sort list to create Regressor (first dep, then con)
dep=sorted(depression, reverse=True)
sex=content['Gender'].tolist()
age=content['Age'].tolist()

# depression dep: 1, con: 0
cov_dep={"vector":dep, "name":"MDD", "centering":0}
#cov_con={"vector":con, "name":"Control", "centering":0}
# sex - dictionary {vector, name, centering} female: 1, male: 0
cov_sex={"vector":sex, "name":"Sex", "centering":0}
#cov_sex={"vector":sex, "name":"Sex", "centering":"No centering"}
# age - dictionary {vector, name, centering}
cov_age={"vector":age, "name":"Age", "centering":0}
#cov_age={"vector":age, "name":"Age", "centering":"No centering"}

In [29]:
print(dep)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [7]:
# control input length of all mreg inputs
print(len(dep_norm + con_norm),
     len(globals_TiCV),
     len(cov_dep['vector']),
     #len(cov_con['vector']),
     len(cov_sex['vector']),
     len(cov_age['vector']))

62 62 62 62 62


In [8]:
# create multiple regression Node
mreg = Node(spm.MultipleRegressionDesign(), name="MultipleRegression")
mreg.base_dir="/VBM/"
mreg.inputs.in_files = dep_norm + con_norm
mreg.inputs.global_calc_values = globals_TiCV
# use proportional global normalization
mreg.inputs.global_normalization = 2
# cut-off value for a-posteriori probability of GM at a given voxel: 5%, [maybe 20%? -> Ashburner VBM Tutorial]
mreg.inputs.threshold_mask_absolute = 0.05
# think of mask image to threshold
#mreg.inputs.explicit_mask_file=
mreg.inputs.covariates = [cov_dep, cov_sex, cov_age]

# make model (design, not estimate) and print results
results = mreg.run()
print(results.outputs)

200220-14:50:05,700 nipype.workflow INFO:
	 [Node] Setting-up "MultipleRegression" in "/VBM/MultipleRegression".
200220-14:50:05,716 nipype.workflow INFO:
	 [Node] Running "MultipleRegression" ("nipype.interfaces.spm.model.MultipleRegressionDesign")
200220-14:50:12,105 nipype.workflow INFO:
	 [Node] Finished "MultipleRegression".

spm_mat_file = /VBM/MultipleRegression/SPM.mat



In [9]:
# specify MReg directory
MReg_dir = "/VBM/MultipleRegression/"

# estimate model
est_mreg = Node(spm.EstimateModel(), name="estimate_MReg")
est_mreg.base_dir = MReg_dir
est_mreg.inputs.spm_mat_file = MReg_dir+"SPM.mat"
est_mreg.inputs.estimation_method = {"Classical":1}

# run estimation and print output
results = est_mreg.run()
print(results.outputs)

200220-14:50:14,975 nipype.workflow INFO:
	 [Node] Setting-up "estimate_MReg" in "/VBM/MultipleRegression/estimate_MReg".
200220-14:50:14,980 nipype.workflow INFO:
	 [Node] Running "estimate_MReg" ("nipype.interfaces.spm.model.EstimateModel")
200220-14:50:36,973 nipype.workflow INFO:
	 [Node] Finished "estimate_MReg".

ARcoef = <undefined>
Cbetas = <undefined>
RPVimage = /VBM/MultipleRegression/estimate_MReg/RPV.nii
SDbetas = <undefined>
SDerror = <undefined>
beta_images = ['/VBM/MultipleRegression/estimate_MReg/beta_0001.nii', '/VBM/MultipleRegression/estimate_MReg/beta_0002.nii', '/VBM/MultipleRegression/estimate_MReg/beta_0003.nii', '/VBM/MultipleRegression/estimate_MReg/beta_0004.nii']
labels = <undefined>
mask_image = /VBM/MultipleRegression/estimate_MReg/mask.nii
residual_image = /VBM/MultipleRegression/estimate_MReg/ResMS.nii
residual_images = <undefined>
spm_mat_file = /VBM/MultipleRegression/estimate_MReg/SPM.mat



In [49]:
# specify MReg directory
MReg_dir = "/VBM/MultipleRegression/"
# specify extended MReg directory
MReg_dir = MReg_dir+"estimate_MReg/"

# collect beta images from Model estimation
beta_img = sorted([i for i in os.listdir(MReg_dir) if re.match(r'^beta.*.nii', i)])
beta_img = [os.path.join(MReg_dir, i) for i in beta_img]

# define contrasts
cont1 = ('Dep > Con','T', ['mean','MDD','Sex','Age'],[0, 1, 0, 0])
cont2 = ('Male > Female','T', ['mean','MDD','Sex','Age'],[0, 0, -1, 0])
cont3 = ('Young > Old','T', ['mean','MDD','Sex','Age'],[0, 0, 0, -1])
contrasts = [cont1,cont2,cont3]

# estimate contrast
est_contrasts = Node(spm.EstimateContrast(), name="estimate_Contrats")
est_contrasts.base_dir = MReg_dir
est_contrasts.inputs.spm_mat_file = MReg_dir+"SPM.mat"
est_contrasts.inputs.residual_image = MReg_dir+"ResMS.nii"
est_contrasts.inputs.beta_images = beta_img
est_contrasts.inputs.contrasts = contrasts


In [50]:
# run estimation and print output
results = est_contrasts.run() 
print(results.outputsuts)

200220-16:14:38,503 nipype.workflow INFO:
	 [Node] Setting-up "estimate_Contrats" in "/VBM/MultipleRegression/estimate_MReg/estimate_Contrats".
200220-16:14:38,513 nipype.workflow INFO:
	 [Node] Running "estimate_Contrats" ("nipype.interfaces.spm.model.EstimateContrast")
200220-16:14:39,999 nipype.workflow WARNING:
	 Storing result file without outputs
200220-16:14:40,2 nipype.workflow WARNING:
	 [Node] Error on "estimate_Contrats" (/VBM/MultipleRegression/estimate_MReg/estimate_Contrats)


RuntimeError: Command:
/opt/spm12-r7219/run_spm12.sh /opt/matlabmcr-2010a/v713 script /VBM/MultipleRegression/estimate_MReg/estimate_Contrats/pyscript_estimatecontrast.m
Standard output:
Warning: No display specified.  You will not be able to display graphics on the screen.
SPM12, version 7219 (standalone)
MATLAB, version 7.10.0.499 (R2010a)
 ___  ____  __  __                                            
/ __)(  _ \(  \/  )                                           
\__ \ )___/ )    (   Statistical Parametric Mapping           
(___/(__)  (_/\/\_)  SPM12 - http://www.fil.ion.ucl.ac.uk/spm/

Executing /VBM/MultipleRegression/estimate_MReg/estimate_Contrats/pyscript_estimatecontrast.m at 20-Feb-2020 16:14:39:
-------------------------------------------------------------------------------------
MATLAB Version 7.10.0.499 (R2010a)
MATLAB License Number: unknown
Operating System: Linux 4.15.0-74-generic #84-Ubuntu SMP Thu Dec 19 08:06:28 UTC 2019 x86_64
Java VM Version: Java 1.6.0_12-b04 with

Testing Zone:

In [16]:
# TODO: think about sorting values of different lists (globals, covariates all assigned to right subjects


True